In [1]:
# https://dacon.io/competitions/official/235930/codeshare/5508?page=1&dtype=recent


In [2]:
import random
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
epochs = 400
lr = 5e-3
batch_size = 1024
momentum = 0.9
weight_decay = 1e-4
seed = 42
NUM_WORKERS = 8

param = {
            'epochs' : epochs,
            'lr' :lr,
            'batch_size' : batch_size,
            'momentum' : momentum,
            'weight_decay' : weight_decay
        }

In [6]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed)    

In [7]:
train_df = pd.read_csv('../dataset/train.csv')
val_df = pd.read_csv('../dataset/val.csv')
train_df = train_df.drop(columns=['ID'])
val_df = val_df.drop(columns=['ID'])
print(train_df.shape)

(113842, 30)


In [8]:
def reduced_mat(df) :
    u, s, v = np.linalg.svd(df, full_matrices=False)
    s[0] = 0
    rdf = pd.DataFrame( u @ np.diag(s) @ v)
    return rdf

In [9]:
train = reduced_mat(train_df)
train.shape

(113842, 30)

In [10]:
val = reduced_mat(val_df.drop(columns=['Class']))
val['Class'] = val_df['Class']
val.shape

(28462, 31)

In [11]:
class CDataset(Dataset):
    def __init__(self, df, eval_mode=False):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [12]:
train_dataset = CDataset(train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=NUM_WORKERS)

val_dataset = CDataset(val, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [13]:
x = next(iter(train_loader))
x.shape

torch.Size([1024, 30])

In [14]:
x, y  = next(iter(val_loader))
x.shape

torch.Size([1024, 30])

In [15]:
y

tensor([0, 0, 0,  ..., 0, 0, 0])

In [16]:
class AutoEncoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        # self.act = nn.ReLU()
        self.act = nn.GELU()
        # self.act = nn.LeakyReLU()
        
        self.dim = 30
        self.hidden1 = 64
        self.hidden2 = 128
        
        self.encoder = nn.Sequential(
            nn.BatchNorm1d(self.dim),
            nn.Linear(self.dim,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.hidden2),
            nn.BatchNorm1d(self.hidden2),
            self.act,
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden2,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.dim),
        )
        
        self.encoder2 = nn.Sequential(
            nn.BatchNorm1d(self.dim),
            nn.Linear(self.dim,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.hidden2),
            nn.BatchNorm1d(self.hidden2),
            self.act,
        )
        
        self.decoder2 = nn.Sequential(
            nn.Linear(self.hidden2,self.hidden1),
            nn.BatchNorm1d(self.hidden1),
            self.act,
            nn.Linear(self.hidden1,self.dim),
        )        
    

          # tied auto encoder
#         self.l1_weight = torch.randn(self.hidden1, self.dim) / torch.sqrt(torch.tensor(self.hidden1))
#         self.l1_bias = torch.zeros(self.hidden1)
        
#         self.l2_weight = torch.randn(self.hidden2, self.hidden1) / torch.sqrt(torch.tensor(self.hidden2))
#         self.l2_bias = torch.zeros(self.hidden2)

#         self.encoder[1].weight = nn.Parameter(self.l1_weight)
#         self.encoder[1].bais = nn.Parameter(self.l1_bias)
#         self.encoder[4].weight = nn.Parameter(self.l2_weight)
#         self.encoder[4].bias = nn.Parameter(self.l2_bias)

        
#         self.decoder[0].weight = nn.Parameter(self.l2_weight.transpose(0,1))
#         self.encoder[0].bais = nn.Parameter(self.l2_bias)
#         self.encoder[3].weight = nn.Parameter(self.l1_weight.transpose(0,1))
#         self.encoder[3].bias = nn.Parameter(self.l1_bias)   
        
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)          
        
    def forward(self, x) :
        
        x = self.encoder(x)
        x1 = self.decoder(x)
        
        x = self.encoder(x1)
        x = self.decoder(x)
        
        return x1, x
        

In [17]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device, **param) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.batch_size = param['batch_size']
        self.epochs = param['epochs']
        self.lr = param['lr']
        
        self.criterion = nn.L1Loss().to(device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                x1, _x = self.model(x)
                
                loss = self.criterion(x, _x)
                loss = self.criterion(x, _x) + 0.5*self.criterion(x, x1)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                
            score = self.validation(self.model, 0.95)
            
            if self.scheduler is not None :
                self.scheduler.step(score)
            
            print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}]')
            # for param_group in self.optimizer.param_groups:
            #     print(param_group['lr'])      
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}] lr [{self.scheduler.get_lr()}]')

            if best_score < score :
                best_score = score
                torch.save(self.model.state_dict(), '../saved/best_model.pth')
            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                _, _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()
                
        return f1_score(true_y, pred_y, average='macro')

In [18]:
model = AutoEncoder()
model.eval()
# optimizer = torch.optim.Adam(params=model.parameters(), lr = param['lr'])

optimizer = torch.optim.SGD(model.parameters(), param['lr'],
                            momentum=param['momentum'],
                            weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
# scheduler = StepLR(optimizer, step_size=50, gamma=0.2)


In [19]:
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device, **param)
# trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)

In [20]:
trainer.fit()

epoch :[0] train loss [1.2751718589237757] val score [0.0010529271374420891]
epoch :[1] train loss [1.0006130465439387] val score [0.0010529271374420891]
epoch :[2] train loss [0.9136153581951346] val score [0.0010529271374420891]
epoch :[3] train loss [0.8589745914297444] val score [0.0010529271374420891]
epoch :[4] train loss [0.8147940981600966] val score [0.0010529271374420891]
epoch :[5] train loss [0.7778674725975309] val score [0.0016862816792256323]
epoch :[6] train loss [0.7449012313570295] val score [0.002108073923125571]
epoch :[7] train loss [0.7167490316288812] val score [0.0035465233735341513]
epoch :[8] train loss [0.6912844175738948] val score [0.00533006134799683]
epoch :[9] train loss [0.6689096588109221] val score [0.0078372014167055]
epoch :[10] train loss [0.6467952781489917] val score [0.010366347561944598]
epoch :[11] train loss [0.6270900044058051] val score [0.013535537740313415]
epoch :[12] train loss [0.6086725566004004] val score [0.020389710236054192]
epoch